In [4]:
!pip install tensorboard


  Obtaining dependency information for tensorboard from https://files.pythonhosted.org/packages/0a/32/2e8545fb0592f33e3aca5951e8b01008b76d61b440658cbdc37b4eaebf0b/tensorboard-2.17.0-py3-none-any.whl.metadata
  Obtaining dependency information for absl-py>=0.4 from https://files.pythonhosted.org/packages/a2/ad/e0d3c824784ff121c03cc031f944bc7e139a8f1870ffd2845cc2dd76f6c4/absl_py-2.1.0-py3-none-any.whl.metadata
  Obtaining dependency information for grpcio>=1.48.2 from https://files.pythonhosted.org/packages/11/8d/049fee0b7374b948d47a0301fb97d71af067ffd5e5d8fbe9190475a76065/grpcio-1.64.1-cp311-cp311-macosx_10_9_universal2.whl.metadata
  Obtaining dependency information for protobuf!=4.24.0,<5.0.0,>=3.19.6 from https://files.pythonhosted.org/packages/f3/bf/26deba06a4c910a85f78245cac7698f67cedd7efe00d04f6b3e1b3506a59/protobuf-4.25.3-cp37-abi3-macosx_10_9_universal2.whl.metadata
  Obtaining dependency information for tensorboard-data-server<0.8.0,>=0.7.0 from https://files.pythonhosted.org/p

In [6]:
# 导入必要的库
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import TensorDataset, DataLoader

In [7]:
# TensorboardWriter函数定义
def TensorboardWriter(model, X, Y, epoch, loss, name):
    Y_pred = model(X)
    CEloss = loss(Y_pred, Y)
    result = torch.max(Y_pred.data, dim=1).indices
    accuracy = result.eq(Y).sum().item() / len(Y_pred)
    writer.add_scalar(f"Loss/{name}_Loss", CEloss, epoch)
    writer.add_scalar(f"Accuracy/{name}_Accuracy", accuracy, epoch)

# 定义单层神经网络模型
class NetWork(nn.Module):
    def __init__(self, input_feature, output):
        super().__init__()
        self.fc1 = nn.Linear(input_feature, output, bias=False)
        nn.init.xavier_normal_(self.fc1.weight)
        self.fc2 = nn.Softmax(dim=1)
    
    def forward(self, x):
        x = self.fc1(x)
        x = self.fc2(x)
        return x

In [8]:
# 加载数据
X_train = torch.load("X_train.pt")
Y_train = torch.load("Y_train.pt")
X_valid = torch.load("X_valid.pt")
Y_valid = torch.load("Y_valid.pt")

# 初始化模型
model = NetWork(300, 4)

# 定义损失函数和优化器
loss = nn.CrossEntropyLoss(reduction="mean")
optimizer = optim.SGD(params=model.parameters(), lr=0.01)

# 创建DataLoader
ds = TensorDataset(X_train, Y_train)
DL = DataLoader(ds, batch_size=128, shuffle=True)

# 初始化TensorBoard的SummaryWriter
writer = SummaryWriter(log_dir="logs")

In [9]:
# 训练模型
epoch = 1000
for ep in range(epoch):
    for X, Y in DL:
        Y_pred = model(X)
        CEloss = loss(Y_pred, Y)
        optimizer.zero_grad()
        CEloss.backward()
        optimizer.step()
    
    # 记录训练和验证的损失和准确率
    TensorboardWriter(model, X_train, Y_train, ep, loss, "Train")
    TensorboardWriter(model, X_valid, Y_valid, ep, loss, "Valid")

# 保存模型
torch.save(model.state_dict(), "SingleLayer.pth")
writer.close()

In [10]:
# 启动TensorBoard
%load_ext tensorboard
%tensorboard --logdir logs

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
